# Project Outline: Minimizing SUMMA Model Misfits at the Cues Site
CVU - Intro to Snow Hydrology: Focus on Modeling

Bareera Mirza & Rainey Aberle

29 October 2021

#### SUMMA parameters / decisions to modify:
1. Downward longwave radiation parameterization
2. `snowIncept`: `stickySnow` vs. `lightSnow`
3. 

#### Set path to the cues folder

In [1]:
homepath = '/home/jovyan/data/work/CVU_SnowHydro_CourseProject/cues/'

#### Import Packages

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt # if you want to do some plotting using matplotlib (optional)
import numpy as np # provide some mathematical functions
import xarray as xr # read, write, manipulate and analyze NetCDF files
from pathlib import Path
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
import warnings
warnings.filterwarnings('ignore')
import pysumma as ps

#### Setup the SUMMA executable and file manager

In [3]:
executable = 'summa.exe'
filemanager = str(homepath+'summa_setup/settings/file_manager_cues.txt')

In [2]:
# Stephan-Boltzmann constant (J/s/m^2/K^4)
STEFAN = 5.67e-8


def vapor_pressure(air_pressure, spec_humid):
    """See above for derivation"""
    return -1.607 * air_pressure * spec_humid / (spec_humid - 1)


def longwave_prata(air_temp, vapor_pressure):
    """
    Reference:
        Prata, A.J., 1996. A new long-wave formula for estimating
        downward clear-sky radiation at the surface. Q. J. R. Meteor.
        Soc. 122 (533), 1127–1151, doi:10.1002/qj.49712253306.
    """
    z = 46.5 * (vapor_pressure / air_temp)
    emissivity = 1 - (1 + z) * np.exp(-np.sqrt(1.2 + 3 * z))
    return emissivity * np.power(air_temp, 4) * STEFAN
        
    
def longwave_tva(air_temp, vapor_pressure):
    """
    Reference:
        Tennessee Valley Authority, 1972. Heat and mass transfer between a
        water surface and the atmosphere. Tennessee Valley Authority, Norris,
        TN. Laboratory report no. 14. Water resources research report no. 0-6803.
    """
    emissivity = 0.74 + 0.0049 * vapor_pressure/10
    return emissivity * np.power(air_temp, 4) * STEFAN


def longwave_satterlund(air_temp, vapor_pressure):
    """
    Reference:
        Satterlund, D.R., 1979. An improved equation for estimating long-wave
        radiation from the atmosphere. Water Resour. Res. 15 (6), 1649–1650,
        doi:10.1029/WR015i006p01649.
    """
    vp = vapor_pressure / 10
    emissivity = 1.08 * (1 - np.exp(-np.power(vp, air_temp/2016)))
    return emissivity * np.power(air_temp, 4) * STEFAN


def longwave_anderson(air_temp, vapor_pressure):
    """
    Referencce:
        Anderson, E.R., 1954. Energy budget studies, water loss
        investigations: lake Hefner studies. U.S. Geol. Surv. Prof. Pap. 269,
        71–119 [Available from U.S. Geological Survey, 807 National Center,
        Reston, VA 20192.].
    """
    emissivity = 0.68 + 0.036 * np.power(vapor_pressure/10, 0.5)
    return emissivity * np.power(air_temp, 4) * STEFAN


def cloud_correction(shortwave, lat=43.03, highlimit=0.6, lowlimit=0.35):
    """Converted from cloudfactor_Jessica.m"""
    doy = shortwave.time.dt.dayofyear
    s0 = 1360                     # Solar constant (W/m^2)
    phi = lat * 2 * np.pi / 365   # Convert to radian
    # Declination in radians
    delta = (2 * np.pi / 365) * (23.45 * np.sin(2 * np.pi * (284 + doy) / 365))
    
    # Top of atmosphere radiation
    hs = np.arccos(-np.tan(phi) * np.tan(delta))
    q0 = s0 * (1/np.pi) * (
        hs * np.sin(phi) * np.sin(delta) + np.cos(phi) * np.cos(delta) * np.sin(hs))
    
    # Fraction of recieved radiation
    k = shortwave / q0
    
    # Cloud cover fraction
    cloud_frac = 1 - ((k - lowlimit) / (highlimit - lowlimit))
    cloud_frac = cloud_frac.where(cloud_frac > 0, other=0.0)
    cloud_frac = cloud_frac.where(cloud_frac < 1, other=1.0)
    return cloud_frac
    
    
def longwave_dokia(air_temp, vapor_pressure, shortwave):
    """
    References:
      -Clear sky:
        Dilley, A. C., and D. M. O<92>Brien (1998), Estimating downward clear sky
        long-wave irradiance at the surface from screen temperature and precipitable
        water, Q. J. R. Meteorol. Soc., 124, 1391<96> 1401.
      -Cloudy sky:
        Kimball, B. A., S. B. Idso, and J. K. Aase (1982), A model of thermal                                                                                          
        radiation from partly cloudy and overcast skies, Water Resour. Res., 18,                                                                                       
        931<96> 936.                                   
    """
    vp = vapor_pressure / 1000 # Convert to kPa
    w = 4560 * (vp / air_temp) # Prata (1996) approximation for precipitable water
    
    # Clear sky component of longwave
    lw_clear = 59.38 + 113.7 * np.power(air_temp / 273.16, 6) + 96.96 * np.sqrt(w / 25)
    
    # Cloud cover corrections
    c = cloud_correction(shortwave)
    cloud_temp = air_temp - 11
    winter = np.logical_or(vp.time.dt.month <= 2, vp.time.dt.month == 12)
    summer = np.logical_and(vp.time.dt.month <= 8, vp.time.dt.month >=6)
    cloud_temp[winter] -= 2
    cloud_temp[summer] += 2
    
    # Cloudy sky component of longwave
    eps8z = 0.24 + 2.98e-6 * np.power(vp, 2.0) * np.exp(3000/air_temp)
    tau8 = 1 - eps8z * (1.4 - (0.4 * eps8z))
    f8 = -0.6732 + 0.6240e-2 * cloud_temp - 0.914e-5 * np.power(cloud_temp, 2.0)
    lw_cloud = tau8 * c * f8 * STEFAN * np.power(cloud_temp, 4)   
    return lw_clear + lw_cloud